## Eliminar un disco de volumen de grupo

Lo que quiere decir es separar un volumen de grupo, lo que quiere decir que se va a separar un volumen de grupo y crear uno nuevo.

Debemos estar en la máquina virtual con los 5 discos sin formato (si sus discos duros tienen formato, favor restaurar la instantánea apropiada)

Logueados como el usuario **root**, nos aseguramos de que tengamos los discos sin formato

`fdisk -l | grep sd`

***
```
root@debian12lvm1:~# fdisk -l | grep sd
Disco /dev/sde: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdf: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdd: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdc: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdb: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sda: 10 GiB, 10737418240 bytes, 20971520 sectores
/dev/sda1  *          2048 18970623 18968576     9G 83 Linux
/dev/sda2         18972670 20969471  1996802   975M  5 Extendida
/dev/sda5         18972672 20969471  1996800   975M 82 Linux swap / Solaris
```
***

Ahora revisamos el script para asegurarse que se formatearan los 5 discos, pero agregaremos otra instrucción al script para crear un volúmen de grupo como se muestra a continuación

`cp 123 1234`

`nano 1234`

y agregar lo siguiente

```
# crear un volumen de grupo
vgcreate newvg /dev/sdb1 /dev/sdc1 /dev/sdd1

lvcreate -n newlv newvg -l 100%FREE
mkfs.ext4 /dev/newvg/newlv
mount /dev/newvg/newlv /mnt

dd if=/dev/urandom of=/mnt/5.txt bs=5M count=1024 status=progress
dd if=/dev/urandom of=/mnt/4.txt bs=4M count=1024 status=progress
dd if=/dev/urandom of=/mnt/1.txt bs=1M count=1024 status=progress
```

quedando el contenido del archivo de la siguiente forma

**1234**

***
```
#!/bin/bash
#Estos comandos Establecen que van a ser discos msdos master boot record 
parted /dev/sdb mklabel msdos
parted /dev/sdc mklabel msdos
parted /dev/sdd mklabel msdos
parted /dev/sde mklabel msdos
parted /dev/sdf mklabel msdos

#Estos comandos crean una particion primaria y le hacen un set a lvm
parted -s /dev/sdb unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdc unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdd unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sde unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdf unit mib mkpart primary 1 100% set 1 lvm on

# Solo si es RHEL8
#Este comando le dice al kernel que reconozca esto nuevo
#udevadm settle

# De lo contrario ejecute
partprobe -s

# crear un volumen de grupo
vgcreate newvg /dev/sdb1 /dev/sdc1 /dev/sdd1

lvcreate -n newlv newvg -l 100%FREE
mkfs.ext4 /dev/newvg/newlv
mount /dev/newvg/newlv /mnt

dd if=/dev/urandom of=/mnt/5.txt bs=5M count=1024 status=progress
dd if=/dev/urandom of=/mnt/4.txt bs=4M count=1024 status=progress
dd if=/dev/urandom of=/mnt/1.txt bs=1M count=1024 status=progress

```
***

Las ultimas tres lineas crean tres archivos de los siguientes tamaños, 5 GB, 4GB y 1 GB para un total de 10 GB.

Ahora procedemos a ejecutar el script **1234**

`sh 1234`

y su salida es 

***
```
root@debian12lvm1:~# sh 1234
Information: You may need to update /etc/fstab.

Information: You may need to update /etc/fstab.                           

Information: You may need to update /etc/fstab.                           

Information: You may need to update /etc/fstab.                           

Information: You may need to update /etc/fstab.                           

/dev/sda: msdos partitions 1 2 <5>                                        
/dev/sdb: msdos partitions 1
/dev/sdc: msdos partitions 1
/dev/sdd: msdos partitions 1
/dev/sde: msdos partitions 1
/dev/sdf: msdos partitions 1
  Physical volume "/dev/sdb1" successfully created.
  Physical volume "/dev/sdc1" successfully created.
  Physical volume "/dev/sdd1" successfully created.
  Volume group "newvg" successfully created
  Logical volume "newlv" created.
mke2fs 1.47.0 (5-Feb-2023)
Creating filesystem with 3929088 4k blocks and 983040 inodes
Filesystem UUID: 1a26ae29-8c04-4460-b203-7d5b012b3662
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912, 819200, 884736, 1605632, 2654208

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (16384 blocks): done
Writing superblocks and filesystem accounting information: done   

5311037440 bytes (5.3 GB, 4.9 GiB) copied, 76 s, 69.9 MB/s
1024+0 records in
1024+0 records out
5368709120 bytes (5.4 GB, 5.0 GiB) copied, 76.8541 s, 69.9 MB/s
4236247040 bytes (4.2 GB, 3.9 GiB) copied, 61 s, 69.4 MB/s
1024+0 records in
1024+0 records out
4294967296 bytes (4.3 GB, 4.0 GiB) copied, 61.8442 s, 69.4 MB/s
1073741824 bytes (1.1 GB, 1.0 GiB) copied, 15 s, 71.5 MB/s
1024+0 records in
1024+0 records out
1073741824 bytes (1.1 GB, 1.0 GiB) copied, 15.014 s, 71.5 MB/s

```
***

Ahora ya tenemos nuestro espacio listo para comenzar a trabajar, y para comenzar este ejercicio vamos a ver los volumenes físicos 

`pvs`

***
```
root@debian12lvm1:~# pvs
  PV         VG    Fmt  Attr PSize  PFree
  /dev/sdb1  newvg lvm2 a--  <5.00g    0 
  /dev/sdc1  newvg lvm2 a--  <5.00g    0 
  /dev/sdd1  newvg lvm2 a--  <5.00g    0
```
***


Ahora agregaremos mas opciones al comando, para ver en dónde se usan los volumenes

`pvs -o+pv_used`

***
```
root@debian12lvm1:~# pvs -o+pv_used
  PV         VG    Fmt  Attr PSize  PFree Used  
  /dev/sdb1  newvg lvm2 a--  <5.00g    0  <5.00g
  /dev/sdc1  newvg lvm2 a--  <5.00g    0  <5.00g
  /dev/sdd1  newvg lvm2 a--  <5.00g    0  <5.00g
```
***

Ahora mostramos que tan grande es el sistema de archivos, **que son 15GB**

`df -h /mnt`

***
```
root@debian12lvm1:~# df -h /mnt
S.ficheros              Tamaño Usados  Disp Uso% Montado en
/dev/mapper/newvg-newlv    15G    11G  3.9G  73%  /mnt

```
***

Aqui esta usandose todo, no podemos sacar información a ningun lado, porque no hay espacio libre. Así que hay que extender el volumen físico

`vgextend newvg /dev/sde1`

***
```
root@debian12lvm1:~# vgextend newvg /dev/sde1
  Physical volume "/dev/sde1" successfully created.
  Volume group "newvg" successfully extended
```
***

Ahora extendendemos el volúmen lógico

`lvextend /dev/newvg/newlv -l +100%FREE -r`

***
```
root@debian12lvm1:~# lvextend /dev/newvg/newlv -l +100%FREE -r
  Size of logical volume newvg/newlv changed from <14.99 GiB (3837 extents) to 19.98 GiB (5116 extents).
  Logical volume newvg/newlv successfully resized.
resize2fs 1.47.0 (5-Feb-2023)
Filesystem at /dev/mapper/newvg-newlv is mounted on /mnt; on-line resizing required
old_desc_blocks = 2, new_desc_blocks = 3
The filesystem on /dev/mapper/newvg-newlv is now 5238784 (4k) blocks long.
```
***

Ahora mostramos que tan grande es el sistema de archivos, que son **20 GB**

`df -h /mnt`

***
```
root@debian12lvm1:~# df -h /mnt
S.ficheros              Tamaño Usados  Disp Uso% Montado en
/dev/mapper/newvg-newlv    20G    11G  8.7G  54% /mnt

```
***

Ahora volvemos a ver los volumenes físicos

`pvs`

***
```
root@debian12lvm1:~# pvs
  PV         VG    Fmt  Attr PSize  PFree
  /dev/sdb1  newvg lvm2 a--  <5.00g    0 
  /dev/sdc1  newvg lvm2 a--  <5.00g    0 
  /dev/sdd1  newvg lvm2 a--  <5.00g    0 
  /dev/sde1  newvg lvm2 a--  <5.00g    0
```
***

Y ahora mostramos información de cuanto esta siendo usado

`pvs -o+pv_used`

***
```
root@debian12lvm1:~# pvs -o+pv_used
  PV         VG    Fmt  Attr PSize  PFree Used  
  /dev/sdb1  newvg lvm2 a--  <5.00g    0  <5.00g
  /dev/sdc1  newvg lvm2 a--  <5.00g    0  <5.00g
  /dev/sdd1  newvg lvm2 a--  <5.00g    0  <5.00g
  /dev/sde1  newvg lvm2 a--  <5.00g    0  <5.00g
```
***

Ahora extendemos el volumen físico agregando al volúmen de grupo a **/dev/sdf1**

`vgextend newvg /dev/sdf1`

***
```
root@debian12lvm1:~# vgextend newvg /dev/sdf1
  Physical volume "/dev/sdf1" successfully created.
  Volume group "newvg" successfully extended

```
***

Y ahora mostramos información de cuanto esta siendo usado

`pvs -o+pv_used`

***
```
root@debian12lvm1:~# pvs -o+pv_used
  PV         VG    Fmt  Attr PSize  PFree  Used  
  /dev/sdb1  newvg lvm2 a--  <5.00g     0  <5.00g
  /dev/sdc1  newvg lvm2 a--  <5.00g     0  <5.00g
  /dev/sdd1  newvg lvm2 a--  <5.00g     0  <5.00g
  /dev/sde1  newvg lvm2 a--  <5.00g     0  <5.00g
  /dev/sdf1  newvg lvm2 a--  <5.00g <5.00g     0 
```
***

Vamos a mover **/dev/sdb1**, entonces, si hay suficientes extensiones libres en los otros volúmenes físicos, en este caso en la línea **/dev/sdf1  newvg lvm2 a--  <5.00g <5.00g     0** podemos ejecutar el comando para sacar un volúmen **/dev/sdb1** de allí con el siguiente comando, esta moviendo los physical extends.

`pvmove /dev/sdb1`

***
```
root@debian12lvm1:~# pvmove /dev/sdb1
  /dev/sdb1: Moved: 0.08%
  /dev/sdb1: Moved: 12.74%
  /dev/sdb1: Moved: 22.52%
  /dev/sdb1: Moved: 31.04%
  /dev/sdb1: Moved: 42.77%
  /dev/sdb1: Moved: 52.54%
  /dev/sdb1: Moved: 62.00%
  /dev/sdb1: Moved: 70.13%
  /dev/sdb1: Moved: 79.28%
  /dev/sdb1: Moved: 91.48%
  /dev/sdb1: Moved: 100.00%

```
***

Una vez que termina, se vuelve a ejecutar el comando 

`pvs -o+pv_used`

***
```
root@debian12lvm1:~# pvs -o+pv_used
  PV         VG    Fmt  Attr PSize  PFree  Used  
  /dev/sdb1  newvg lvm2 a--  <5.00g <5.00g     0 
  /dev/sdc1  newvg lvm2 a--  <5.00g     0  <5.00g
  /dev/sdd1  newvg lvm2 a--  <5.00g     0  <5.00g
  /dev/sde1  newvg lvm2 a--  <5.00g     0  <5.00g
  /dev/sdf1  newvg lvm2 a--  <5.00g     0  <5.00g

```
***

Como se puede observar **/dev/sdb1** ya tiene todo libre, ya fue movido todo para el volúmen físico **/dev/sdf1** 

Ahora, el siguiente paso es **reducir** el volúmen de grupo

`vgreduce newvg /dev/sdb1`

***
```
root@debian12lvm1:~# vgreduce newvg /dev/sdb1
  Removed "/dev/sdb1" from volume group "newvg"

```
***

Y al ejecutar el siguiente comando para ver los volumenes físicos

`pvs`

***
```
root@debian12lvm1:~# pvs
  PV         VG    Fmt  Attr PSize  PFree 
  /dev/sdb1        lvm2 ---  <5.00g <5.00g
  /dev/sdc1  newvg lvm2 a--  <5.00g     0 
  /dev/sdd1  newvg lvm2 a--  <5.00g     0 
  /dev/sde1  newvg lvm2 a--  <5.00g     0 
  /dev/sdf1  newvg lvm2 a--  <5.00g     0 
```
***

Podemos ver que **/dev/sdb1** no pertenece a ningún volúmen de grupo *VG* y puede ser físicamente eliminado, es decir, se puede extraer el disco, cambiado o asignado a otros usuario, proyectos, etc.


